In [11]:
import os
import json
import csv

In [12]:
def get_prompts(filename):
    with open(filename) as f:
        prompts = []
        for line in f:
            prompts.append(json.loads(line))
    return prompts


In [13]:
result = {}
fix_keys = {}
benchmark_root = "../Repair_Quality_Analyzer/Repair_Quality_Filtered_Suggestions/"
dir_list = os.listdir(benchmark_root)
for file in dir_list:
    if ".DS_Store" in file:
        continue
    print("Processing file: ", file)
    result[file] = {}
    prompts = get_prompts(benchmark_root + file)
    print("Number of prompts: ", len(prompts))
    if len(prompts) == 0:
        continue
    for prompt in prompts:
        to_be_fixed = []
        check_not_fixed = {}
        for line in prompt["repair_prompt"].split("\n"):
            if line.startswith("# Fix: "):
                # print(",".join(line.split(", ")[1:]))
                fix_key = ",".join(line.split(", ")[1:])
                if fix_key not in fix_keys:
                    fix_keys[fix_key] = {
                        "total": 1,
                        "fixed_prompt": 0,
                        "fixed_suggestion": 0,
                    }
                else:
                    fix_keys[fix_key]["total"] += 1

                if fix_key not in check_not_fixed:
                    check_not_fixed[fix_key] = 10
                to_be_fixed.append(",".join(line.split(", ")[1:]))
        if len(to_be_fixed) > 0:
            result[file][prompt["task_id"]] = to_be_fixed
        else:
            continue
        for issue_text in check_not_fixed:
            total = 0
            for i in range(len(prompt["suggestions"])):
                suggestion = prompt["suggestions"][i]
                found = False
                if not suggestion["Is_Compilable"]:
                    check_not_fixed[issue_text] -= 1
                    continue
                for issue in suggestion["Analyzer_Result"]:
                    if issue["issue_text"] in issue_text:
                        found = True
                        break
                if found:
                    total += 1
                
            check_not_fixed[issue_text] = check_not_fixed[issue_text] - total
                    
        for key in check_not_fixed:
            if check_not_fixed[key] >0:
                fix_keys[key]["fixed_prompt"] += 1
            fix_keys[key]["fixed_suggestion"] += check_not_fixed[key]

    if len(result[file]) == 0:
        del result[file]

print("Number of files: ", len(result))
print("Number of fix keys: ", fix_keys)


Processing file:  SecurityEval_python_codeparrot-small_128_10.jsonl
Number of prompts:  14
Processing file:  SOEvalJava_PolyCoder-0.4B_128_10.jsonl
Number of prompts:  1
Processing file:  CoderEval4Python_prompt_codeparrot_128_10.jsonl
Number of prompts:  0
Processing file:  SOEvalPython_codegen-350M-mono_128_10.jsonl
Number of prompts:  1
Processing file:  HumanEval_java_incoder-1B_128_10.jsonl
Number of prompts:  15
Processing file:  SecurityEval_python_codegen-2B-multi_128_10.jsonl
Number of prompts:  16
Processing file:  HumanEval_python_PolyCoder-2.7B_128_10.jsonl
Number of prompts:  0
Processing file:  SOEvalJava_codegen-2B-multi_128_10.jsonl
Number of prompts:  1
Processing file:  CoderEval4Python_prompt_codegen-350M-multi_128_10.jsonl
Number of prompts:  0
Processing file:  SecurityEval_python_codegen-2B-mono_128_10.jsonl
Number of prompts:  13
Processing file:  CoderEval4Java_prompt_PolyCoder-0.4B_128_10.jsonl
Number of prompts:  9
Processing file:  HumanEval_python_codegen-35

In [14]:
for key in fix_keys:
    fix_keys[key]["Percentage_fixed_prompt"] = (fix_keys[key]["fixed_prompt"] / fix_keys[key]["total"])*100

In [15]:
import pandas as pd

df = pd.DataFrame.from_dict(fix_keys)
df.to_csv("repair_discussion_result.csv")